In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix,MeshcatVisualizer, StartMeshcat
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.all import PointCloud, MeshcatVisualizerParams, Role, HalfSpace, CoulombFriction, Box, Rgba
from pydrake.polynomial import Polynomial as PolynomialCommon
import time
from pydrake.all import AddFrameTriadIllustration


from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath
from tqdm import tqdm


In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

In [5]:
meshcat = StartMeshcat()

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:cwd = "/home/amice/Documents/coding_projects/drake/C_Iris_Examples"
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_rrt.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/geometry/meshcat.js' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_rrt.runfiles/drake/geometry/meshcat.js' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [6]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/iiwa6/iiwa6_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)

height = 0.1
ground_box = Box(10,10,0.1)
ground_offset = RigidTransform(p=[0,0,-height/2])
friction = CoulombFriction(1,1)
ground_collision = plant.RegisterCollisionGeometry(
    plant.world_body(), ground_offset, ground_box, "ground_col", friction
)
# ground_visual = plant.RegisterVisualGeometry(
#     plant.world_body(), ground_offset, ground_box, "ground", [1,0,0,0]
# )


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0
x_fac = 0.7
z_offset = 0.0
locs = [ [0,0,0], 
        [x_fac, 1.4*sp,z_offset],
        [x_fac,-1.4*sp,z_offset],
        [-x_fac,-1.4*sp,z_offset],
        [-x_fac,1.4,z_offset], 
        [0.0 ,0 , z_offset],
        [0.0 ,0 , z_offset]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1
    
# plant.WeldFrames(plant.GetBodyByName("iiwa_link_7").body_frame(), 
#                  plant.GetBodyByName("iiwa_link_6").body_frame(), 
#                  RigidTransform())


plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

meshcat.Delete()
visualizer_params = MeshcatVisualizerParams()
visualizer_params.role = Role.kIllustration #Role.kProximity
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, visualizer_params)
diagram = builder.Build()

AddFrameTriadIllustration(scene_graph = scene_graph,
                        body = plant.get_body(plant.GetBodyIndices(models[-1])[0]))


diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
scene_graph_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_context)
query_port = scene_graph.get_query_output_port()
diagram.ForcedPublish(diagram_context)

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_rrt.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/C_Iris_Examples/assets/iiwa6/iiwa6_with_box_collision.sdf' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/7dof_iiwas_rrt.runfiles/drake/C_Iris_Examples/assets/iiwa6/iiwa6_with_box_collision.sdf' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (goo

In [7]:
plant.num_positions()

6

In [8]:
#compute limits in s-space
limits_s = []
lower_q = plant.GetPositionLowerLimits()
upper_q = plant.GetPositionUpperLimits()

lower_q[0] = -np.pi/2
upper_q[0] = np.pi/2

lower_q[-1] = 0
upper_q[-1] = 0

lower_q[2] = 0

for q in [lower_q,upper_q]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)
print(limits_s)

[[ -1.          -1.7320606    0.          -1.7320606  -11.43007018
    0.        ]
 [  1.           1.7320606   11.43007018   1.7320606   11.43007018
    0.        ]]


In [9]:
from pydrake.all import InverseKinematics
ik = InverseKinematics(plant, diagram.GetMutableSubsystemContext(plant, diagram_context))
min_dist = 1e-5
collision_constraint = ik.AddMinimumDistanceConstraint(
            min_dist, 1e-5)
diagram_col_context = diagram.CreateDefaultContext()
plant_col_context = diagram.GetMutableSubsystemContext(plant, diagram_col_context)
scene_graph_col_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_col_context)
def check_collision_q_by_ik(q, min_dist=1e-5):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        return 1 - 1 * \
            float(collision_constraint.evaluator().CheckSatisfied(q, min_dist))
    else:
        return 1
def check_collision_s_by_ik(s, min_dist=1e-5):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_ik(q, min_dist)

def check_collision_q_by_query(q):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        plant.SetPositions(plant_col_context, q)
        query_object = query_port.Eval(scene_graph_col_context)
        return 1 if query_object.HasCollisions() else 0
    else:
        return 1
    
def check_collision_s_by_query(s):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_query(q)

In [10]:
# prm_important_samples_q = np.array([
# #     np.zeros(plant.num_positions()),
#     [-0.47, -0.19, 0.34, -1.89, 0.68, 0.18, 0],
#     [-2.1, -1.4,  1.53, -1.9 , -2.77,  0.6 ,  0],
#     [2.22, -2.09, -1.61, -1.61, -2.87,  0.64,  0],
#     [0.39,  0.41, -0.17, -1.79,  1.13, -0.49,  0]
# ]) 


prm_important_samples_q = np.array([
    np.zeros(plant.num_positions()),
    [ 0.033,  0.306, -0.067, -0.894,  0.033,  0.006],
    [ 0.033,  0.706, -0.367, -0.894,  0.033,  0.506],
    [-0.267,  1.106,  1.633, -0.894,  0.433, -0.894],
    [-0.467,  1.806,  1.833, -0.694,  0.433, -0.894],
    [-1.367,  1.606, -1.667,  2.094, -0.867, -0.094]
]) 


prm_important_samples_s = [ Ratfk.ComputeSValue(x, q_star) for x in prm_important_samples_q]

for s in prm_important_samples_s:
    print(check_collision_s_by_query(s))

0
0
0
0
0
0


In [11]:
def visualize_task_space_trajectory(segment_start, segment_end,
                                    body, color = Rgba(0,1,0,0.5),
                                    path_size = 0.01, num_points = 100, path = "prm/seg"):
    s_waypoints = np.linspace(segment_start, segment_end, num_points)
    points = []
    for s in s_waypoints:
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        points.append(plant.EvalBodyPoseInWorld(plant_context, body).translation())
    points = np.array(points)
    pc = PointCloud(len(points))
    pc.mutable_xyzs()[:] = points.T
    meshcat.SetObject(path, pc, point_size = path_size, rgba=color)

In [12]:
from scipy.spatial import KDTree
import networkx as nx
from pydrake.all import Sphere, Rgba 
from tqdm import tqdm

class Node:
    def __init__(self, pos, parent=None, children = None):
        self.pos = pos
        self.parent = parent
        self.children = [] if children is None else children
         
class StraightLineCollisionChecker:
    def __init__(self, in_collision_handle, query_density = 100):
        self.in_collision_handle = in_collision_handle
        self.query_density = query_density

    def straight_line_has_collision(self, start, end):
        for pos in np.linspace(start, end, self.query_density):
            if self.in_collision_handle(pos):
                return True
        return False

class RRT:
    def __init__(self, start_pos, end_pos, lower_limits, 
                 upper_limits, straight_line_col_checker: StraightLineCollisionChecker,
                do_build_max_iter = -1):
        """
        start_pos: start of the rrt
        end_pos: end of the rrt
        lower_limits]upper limits: search in the box [lower_limits, upper_limits]
        collision_check_handle: return True if the configuration is in collision, false otherwise.
        """
        self.tree = nx.Graph()
        self.start_pos = start_pos
        self.end_pos = end_pos
        self.start_node = self.tree.add_node(start_pos)
#         self.end_node = self.tree.add_node(end_pos)
        

        self.lower_limits = lower_limits
        self.upper_limits = upper_limits


        self.straight_line_col_checker = straight_line_col_checker
        if do_build_max_iter > 0:
            self.build_tree(do_build_max_iter)

    def get_nearest_node(self, pos):
        nearest_node = None
        nearest_distance = np.inf
        for node in self.tree.nodes():
            if (dist := np.linalg.norm(node - pos)) < nearest_distance:
                nearest_node = node
                nearest_distance = dist
        return nearest_node, nearest_distance
    

    def get_random_node(self, bisection_tol = 1e-5):
        if np.random.rand() > 0.1:
            pos = np.random.uniform(self.lower_limits, self.upper_limits)
            while self.straight_line_col_checker.in_collision_handle(pos):
                pos = np.random.uniform(self.lower_limits, self.upper_limits)
        else:
            pos = np.array(self.end_pos)
        return pos
                
    def add_node(self, pos, bisection_tol = 1e-5):
        nearest_node, nearest_neighbor_dist = self.get_nearest_node(pos)
        nearest_nod_arr = np.array(nearest_node)
        # run bisection search to extend as far as possible in this direction
        v = pos-nearest_node
        t_upper_bound = 1
        t_lower_bound = 0
        max_extend = False
        if self.straight_line_col_checker.straight_line_has_collision(pos, nearest_nod_arr):
            while t_upper_bound - t_lower_bound > bisection_tol:
                t = (t_upper_bound + t_lower_bound)/2
                cur_end = (1-t)*nearest_nod_arr + t * pos 
                if self.straight_line_col_checker.straight_line_has_collision(cur_end, nearest_nod_arr):
                    t_upper_bound = t
                else:
                    t_lower_bound = t
        else:
            cur_end = pos
            max_extend = True
        new_node = tuple(cur_end)
        self.tree.add_node(new_node)
        self.tree.add_edge(nearest_node, new_node, weight = np.linalg.norm(nearest_nod_arr-cur_end))
        return max_extend
        
    def add_new_random_node(self, bisection_tol = 1e-5):
        pos = self.get_random_node(bisection_tol)
        return add_node(pos, bisection_tol)
        

    def build_tree(self, max_iter = int(1e4), bisection_tol = 1e-5, verbose = True):
        for i in tqdm(range(max_iter)):
            self.add_new_random_node(bisection_tol)
            if self.end_pos in self.tree.nodes():
#             if nx.has_path(self.tree, self.start_pos, self.end_pos):
                return True#nx.has_path(self.tree, self.start_node, self.end_node)
        return False

    def draw_tree(self, meshcat_instance, body, prefix = "rrt", edge_color = Rgba(0,0,1,0.5), 
                  start_color = Rgba(0,1,0,0.5), end_color = Rgba(1,0,0,0.5), 
                  path_size = 0.01, num_points = 100, start_end_radius = 0.05 ):
        meshcat.Delete(prefix)
        for idx, (s0, s1) in enumerate(self.tree.edges()):
            visualize_task_space_trajectory(s0, s1, body, edge_color,
                                            path_size=path_size, num_points=num_points, 
                                            path=f"{prefix}/seg_{idx}")
        start_name = f"{prefix}/start"
        s = self.start_pos
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        start_task_space = plant.EvalBodyPoseInWorld(plant_context, body).translation()
        meshcat_instance.SetObject(start_name,
                               Sphere(start_end_radius),
                               start_color)
        meshcat_instance.SetTransform(start_name, RigidTransform(p=start_task_space))
        
        end_name = f"{prefix}/end"
        s = self.end_pos
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        end_task_space = plant.EvalBodyPoseInWorld(plant_context, body).translation()
        meshcat_instance.SetObject(end_name,
                               Sphere(start_end_radius),
                               end_color)
        meshcat_instance.SetTransform(end_name, RigidTransform(p=end_task_space))


In [13]:
class BiRRT:
    def __init__(self, start_pos, end_pos, lower_limits, 
                 upper_limits, straight_line_col_checker: StraightLineCollisionChecker):
        """
        start_pos: start of the rrt
        end_pos: end of the rrt
        lower_limits]upper limits: search in the box [lower_limits, upper_limits]
        collision_check_handle: return True if the configuration is in collision, false otherwise.
        """
        self.tree_to_start = RRT(start_pos, end_pos, lower_limits, 
                 upper_limits, straight_line_col_checker)
        self.tree_to_end = RRT(end_pos, start_pos, lower_limits, 
                 upper_limits, straight_line_col_checker)
        self.start_pos = start_pos
        self.end_pos = end_pos
#         self.start_node = self.tree.add_node(start_pos)
#         self.end_node = self.tree.add_node(end_pos)
        

        self.lower_limits = lower_limits
        self.upper_limits = upper_limits


        self.straight_line_col_checker = straight_line_col_checker
        
        self.connected_tree = None
        
    def add_node(self, pos, bisection_tol = 1e-5):
        if np.random.rand() > 0.1:
            return (self.tree_to_start.add_node(pos, bisection_tol) and 
                    self.tree_to_end.add_node(pos, bisection_tol))
        else:
            return (self.tree_to_start.add_node(np.array(self.end_pos), bisection_tol) or
            self.tree_to_end.add_node(np.array(self.start_pos), bisection_tol))
    
    def get_random_node(self):
        pos = np.random.uniform(self.lower_limits, self.upper_limits)
        while self.straight_line_col_checker.in_collision_handle(pos):
            pos = np.random.uniform(self.lower_limits, self.upper_limits)
        return pos
    
    def build_tree(self, max_iter = int(1e4), bisection_tol = 1e-5, verbose = True):
        for i in tqdm(range(max_iter)):
            pos = self.get_random_node()
            trees_connected = self.add_node(pos, bisection_tol)
            if trees_connected:
                return True
        return False
    
    def draw_tree(self, meshcat_instance, body, prefix = "bi_rrt",
                  start_tree_edge_color = Rgba(0,1,0,0.5), 
                  start_color = Rgba(0,1,0,0.5), 
                  end_tree_edge_color = Rgba(1,0,0,0.5), 
                  end_color = Rgba(1,0,0,0.5), 
                  path_size = 0.01, num_points = 100, start_end_radius = 0.05 ):
        self.tree_to_start.draw_tree(meshcat_instance, body,
                                    prefix+"/start_tree",
                                    edge_color = start_tree_edge_color,
                                    path_size=path_size, num_points=num_points, 
                                    start_end_radius=start_end_radius)
        self.tree_to_end.draw_tree(meshcat_instance, body,
                                    prefix+"/end_tree",
                                    edge_color = end_tree_edge_color,
                                    path_size=path_size, num_points=num_points, 
                                    start_end_radius=start_end_radius)
        
        start_name = f"{prefix}/start"
        meshcat_instance.SetObject(start_name,
                               Sphere(start_end_radius),
                               start_color)
        
        end_name = f"{prefix}/end"
        meshcat_instance.SetObject(end_name,
                               Sphere(start_end_radius),
                               end_color)
    

In [14]:
# col_checker = StraightLineCollisionChecker(check_collision_s_by_ik, 100)
# bi_rrt = BiRRT(tuple(prm_important_samples_s[0]), 
#               tuple(prm_important_samples_s[1]),
#               limits_s[0], limits_s[1], col_checker)
# connected = bi_rrt.build_tree(1000)
# print(connected)
# bi_rrt.draw_tree(meshcat, plant.GetBodyByName("iiwa_link_7"))

In [25]:
from itertools import combinations, permutations
from joblib import Parallel, delayed

col_checker = StraightLineCollisionChecker(check_collision_s_by_ik, 100)
# trees = Parallel(n_jobs=2)(delayed(RRT)((r1, r2, limits_s[0], limits_s[1], col_checker, 100))
#                            for (r1, r2) in combinations(prm_important_samples_s,2))
trees =[]
for i, (r1, r2) in enumerate(combinations(prm_important_samples_s,2)):
# for i, r2 in enumerate(prm_important_samples_s[1:]):
    r1 = prm_important_samples_s[0]
    trees.append(BiRRT(tuple(r1), tuple(r2), limits_s[0], limits_s[1], col_checker))
    tree_connected = trees[-1].build_tree(max_iter = int(1e2))
    print(tree_connected)
    if True: #tree_connected:
        trees[-1].draw_tree(meshcat, plant.GetBodyByName("iiwa_link_7"), prefix = f"rrt/{i}")

 11%|████▌                                     | 11/100 [00:00<00:03, 22.53it/s]


True


100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 43.84it/s]


False


100%|█████████████████████████████████████████| 100/100 [00:02<00:00, 34.32it/s]


...

False


  0%|                                                   | 0/100 [00:00<?, ?it/s]


True


 11%|████▌                                     | 11/100 [00:00<00:02, 35.36it/s]


True


In [30]:
tmp = trees[0]
r1 = tmp.tree_to_start
r2 = tmp.tree_to_end

tmp3 = r1.tree | r2.tree

TypeError: unsupported operand type(s) for |: 'Graph' and 'Graph'

In [ ]:
tmp = nx.read_gpickle("rrt_1_1.gpickle")
print(tmp.nodes["start"])

In [ ]:
class IK_PRM:
    def __init__(self, nominal_pos,
                 lower_limits, 
                 upper_limits,
                 straight_line_col_checker: StraightLineCollisionChecker,
                 do_build_max_iter = -1):
        """
        start_pos: start of the rrt
        end_pos: end of the rrt
        lower_limits]upper limits: search in the box [lower_limits, upper_limits]
        collision_check_handle: return True if the configuration is in collision, false otherwise.
        """
        self.tree = nx.Graph()
        self.nominal_pos = nominal_pos
        self.start_node = self.tree.add_node(nominal_pos)
#         self.end_node = self.tree.add_node(end_pos)
        

        self.lower_limits = lower_limits
        self.upper_limits = upper_limits


        self.straight_line_col_checker = straight_line_col_checker
        if do_build_max_iter > 0:
            self.build_tree(do_build_max_iter)

    def get_k_nearest_node(self, pos, k):
        nearest_nodes = [None for _ in range(k)]
        nearest_distances = [np.inf for _ in range(k)]
        for node in self.tree.nodes():
            if (dist := np.linalg.norm(node - pos)) < max(nearest_distance):
                nearest_node[-1] = node
                nearest_distance[-1] = dist
                
                sort(nearest_node, key = lambda x)
                
        return nearest_node, nearest_distance
    

    def get_random_node(self, bisection_tol = 1e-5):
        if np.random.rand() > 0.1:
            pos = np.random.uniform(self.lower_limits, self.upper_limits)
            while self.straight_line_col_checker.in_collision_handle(pos):
                pos = np.random.uniform(self.lower_limits, self.upper_limits)
        else:
            pos = np.array(self.end_pos)
        return pos
                
    def add_node(self, pos, bisection_tol = 1e-5):
        nearest_node, nearest_neighbor_dist = self.get_nearest_node(pos)
        nearest_nod_arr = np.array(nearest_node)
        # run bisection search to extend as far as possible in this direction
        v = pos-nearest_node
        t_upper_bound = 1
        t_lower_bound = 0
        max_extend = False
        if self.straight_line_col_checker.straight_line_has_collision(pos, nearest_nod_arr):
            while t_upper_bound - t_lower_bound > bisection_tol:
                t = (t_upper_bound + t_lower_bound)/2
                cur_end = (1-t)*nearest_nod_arr + t * pos 
                if self.straight_line_col_checker.straight_line_has_collision(cur_end, nearest_nod_arr):
                    t_upper_bound = t
                else:
                    t_lower_bound = t
        else:
            cur_end = pos
            max_extend = True
        new_node = tuple(cur_end)
        self.tree.add_node(new_node)
        self.tree.add_edge(nearest_node, new_node, weight = np.linalg.norm(nearest_nod_arr-cur_end))
        return max_extend
        
    def add_new_random_node(self, bisection_tol = 1e-5):
        pos = self.get_random_node(bisection_tol)
        return add_node(pos, bisection_tol)
        

    def build_tree(self, max_iter = int(1e4), bisection_tol = 1e-5, verbose = True):
        for i in tqdm(range(max_iter)):
            self.add_new_random_node(bisection_tol)
            if self.end_pos in self.tree.nodes():
#             if nx.has_path(self.tree, self.start_pos, self.end_pos):
                return True#nx.has_path(self.tree, self.start_node, self.end_node)
        return False

    def draw_tree(self, meshcat_instance, body, prefix = "rrt", edge_color = Rgba(0,0,1,0.5), 
                  start_color = Rgba(0,1,0,0.5), end_color = Rgba(1,0,0,0.5), 
                  path_size = 0.01, num_points = 100, start_end_radius = 0.05 ):
        meshcat.Delete(prefix)
        for idx, (s0, s1) in enumerate(self.tree.edges()):
            visualize_task_space_trajectory(s0, s1, body, edge_color,
                                            path_size=path_size, num_points=num_points, 
                                            path=f"{prefix}/seg_{idx}")
        start_name = f"{prefix}/start"
        s = self.start_pos
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        start_task_space = plant.EvalBodyPoseInWorld(plant_context, body).translation()
        meshcat_instance.SetObject(start_name,
                               Sphere(start_end_radius),
                               start_color)
        meshcat_instance.SetTransform(start_name, RigidTransform(p=start_task_space))
        
        end_name = f"{prefix}/end"
        s = self.end_pos
        q = Ratfk.ComputeQValue(s, q_star)
        plant.SetPositions(plant_context, q)
        end_task_space = plant.EvalBodyPoseInWorld(plant_context, body).translation()
        meshcat_instance.SetObject(end_name,
                               Sphere(start_end_radius),
                               end_color)
        meshcat_instance.SetTransform(end_name, RigidTransform(p=end_task_space))


In [23]:
l1 = ["1", "7", "6", "4", "2"]
l2 = [3,1,5,7,2]



TypeError: list indices must be integers or slices, not str

## Set up the sliders so we can move the plant around manually

In [ ]:
sliders = []
q = np.zeros(plant.num_positions())#prm_important_samples_q[-1].copy()
# print(check_collision_q_by_query(q))
plant.SetPositions(plant_context, q)
diagram.ForcedPublish(diagram_context)
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=q[i], description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

#np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)
#     print(check_collision_q_by_query(q))
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

In [ ]:
print(repr(np.round(plant.GetPositions(plant_context),3)))

DEBUG:drake:Meshcat connection opened from 0000:0000:0000:0000:0000:0000:0000:0001
